In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from pandas.tseries.offsets import BusinessDay
import pandas_datareader.data as web

In [2]:
sp = yf.download('^SPX')

sp.index = pd.to_datetime(sp.index)

# Filter the DataFrame
start_date = '2007-01-01'
end_date = '2023-09-01'
filtered_sp = sp.loc[start_date:end_date]
# Remove the 'Ticker' level from the MultiIndex
filtered_sp.columns = filtered_sp.columns.droplevel(1)
filtered_sp = pd.DataFrame(filtered_sp['Close'], index=filtered_sp.index)
filtered_sp.index = pd.to_datetime(filtered_sp.index).tz_localize(None)

[*********************100%***********************]  1 of 1 completed


In [3]:
training_start_date = '2007-01-01'
training_end_date = '2015-12-31'
test_start_date = '2016-01-01'
test_end_date = end_date

In [4]:
# Fetch the Fama-French 3-factor data
ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)

# Convert PeriodIndex to DatetimeIndex
rf_data = ff_data[0]  # Extract the first dataframe
rf_data.index = rf_data.index.to_timestamp()  # Convert PeriodIndex to Timestamp

# Select the RF column (risk-free rate)
risk_free_rate = pd.DataFrame(rf_data['RF'])

# Resample to daily frequency and forward-fill missing values
risk_free_rate = risk_free_rate.resample('D').ffill()
risk_free_rate = risk_free_rate * 12 / 100
risk_free_rate

C:\Users\benj0\AppData\Local\Temp\ipykernel_90480\656172154.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)
C:\Users\benj0\AppData\Local\Temp\ipykernel_90480\656172154.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)


,RF
Date,
2007-01-01,0.0528
2007-01-02,0.0528
2007-01-03,0.0528
2007-01-04,0.0528
2007-01-05,0.0528
...,...
2023-08-28,0.0540
2023-08-29,0.0540
2023-08-30,0.0540


In [5]:
market_df = filtered_sp.merge(risk_free_rate, left_index=True, right_index=True)

# Create a new business day index
new_index = pd.bdate_range(start=start_date, end=end_date)

# Create a new DataFrame with the business day index
new_market_df = pd.DataFrame(index=new_index)

# Merge the existing market_df with the new DataFrame
new_market_df = new_market_df.merge(market_df, left_index=True, right_index=True, how='left')

# Forward fill the missing values
new_market_df.ffill(inplace=True)

# Rename the index to 'Date' if necessary
new_market_df.index.name = 'Date'

# Display the updated DataFrame
new_market_df

,Close,RF
Date,,
2007-01-01,NaN,NaN
2007-01-02,NaN,NaN
2007-01-03,1416.599976,0.0528
2007-01-04,1418.339966,0.0528
2007-01-05,1409.709961,0.0528
...,...,...
2023-08-28,4433.310059,0.0540
2023-08-29,4497.629883,0.0540
2023-08-30,4514.870117,0.0540


In [22]:
new_market_df.loc['2015-12-31']

Close    2043.939941
RF          0.001200
Name: 2015-12-31 00:00:00, dtype: float64

In [6]:
data = pd.read_csv('filtered_data2.csv')
data.set_index('Date', inplace=True)
data_train = data[training_start_date:training_end_date]
option_df = data_train
option_df

,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness
Date,,,,,,,,,,,,,,
2007-01-03,2007-01-05,C,1410.0,8.60,9.60,510,0.114990,0.721566,0.027841,32351194,100,2,1416.599976,0.995341
2007-01-03,2007-01-05,C,1425.0,0.85,1.15,452,0.088166,0.195586,0.029876,32351201,100,2,1416.599976,1.005930
2007-01-03,2007-01-05,P,1410.0,1.65,2.15,398,0.109702,-0.269139,0.028697,32351196,100,2,1416.599976,0.995341
2007-01-03,2007-01-05,P,1425.0,8.50,9.40,121,0.087117,-0.807273,0.029966,32351190,100,2,1416.599976,1.005930
2007-01-03,2007-01-20,C,1410.0,16.20,17.80,853,0.096310,0.640082,0.013096,32267913,100,17,1416.599976,0.995341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31,2016-02-05,P,1920.0,12.10,12.50,96,0.211088,-0.165773,0.001837,109720047,100,36,2043.939941,0.939362
2015-12-31,2016-02-05,P,1925.0,12.70,13.10,49,0.209026,-0.173527,0.001910,109720048,100,36,2043.939941,0.941808
2015-12-31,2016-02-05,P,1940.0,14.70,15.10,13,0.202866,-0.199030,0.002143,109720051,100,36,2043.939941,0.949147


In [7]:
data_train['Midprice'] = (data_train['best_bid'] + data_train['best_offer']) / 2

def calendar_to_business_days(start_date, end_date, calendar_days):

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    # Ensure start_date is in string format
    start_date_str = start_date.strftime('%Y-%m-%d')
    # Calculate the end date
    end_date_str = end_date.strftime('%Y-%m-%d')
    # Calculate business days
    business_days = np.busday_count(start_date_str, end_date_str)
    return business_days

# Apply the function to the 'D to Expiration' column
option_df['Business_Days_to_Expiration'] = option_df.apply(lambda row: calendar_to_business_days(row.name, row['exdate'], row['D to Expiration']), axis=1)

C:\Users\benj0\AppData\Local\Temp\ipykernel_90480\564334160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['Midprice'] = (data_train['best_bid'] + data_train['best_offer']) / 2
C:\Users\benj0\AppData\Local\Temp\ipykernel_90480\564334160.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  option_df['Business_Days_to_Expiration'] = option_df.apply(lambda row: calendar_to_business_days(row.name, row['exdate'], row['D to Expiration']), axis=1)


In [8]:
data_1m = data_train[data_train['Business_Days_to_Expiration'] >= 22]
data_1m.index = pd.to_datetime(data_1m.index)
data_1m

,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness,Midprice,Business_Days_to_Expiration
Date,,,,,,,,,,,,,,,,
2007-01-09,2007-02-17,C,1425.0,14.6,16.2,1566,0.099818,0.455103,0.008711,32304269,100,39,1412.109985,1.009128,15.40,29
2007-01-09,2007-02-17,C,1430.0,12.2,13.8,221,0.098095,0.410637,0.008697,32309398,100,39,1412.109985,1.012669,13.00,29
2007-01-09,2007-02-17,C,1435.0,10.0,11.6,184,0.096213,0.365685,0.008576,32309402,100,39,1412.109985,1.016210,10.80,29
2007-01-09,2007-02-17,C,1440.0,8.3,9.3,220,0.094129,0.320593,0.008341,32303409,100,39,1412.109985,1.019751,8.80,29
2007-01-09,2007-02-17,C,1445.0,6.6,7.6,61,0.092452,0.277198,0.007950,32303408,100,39,1412.109985,1.023291,7.10,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31,2016-02-05,P,1920.0,12.1,12.5,96,0.211088,-0.165773,0.001837,109720047,100,36,2043.939941,0.939362,12.30,26
2015-12-31,2016-02-05,P,1925.0,12.7,13.1,49,0.209026,-0.173527,0.001910,109720048,100,36,2043.939941,0.941808,12.90,26
2015-12-31,2016-02-05,P,1940.0,14.7,15.1,13,0.202866,-0.199030,0.002143,109720051,100,36,2043.939941,0.949147,14.90,26


In [9]:
data_1m.loc['2007-07-10']

,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,delta,gamma,optionid,contract_size,D to Expiration,Close,Moneyness,Midprice,Business_Days_to_Expiration
Date,,,,,,,,,,,,,,,,
2007-07-10,2007-08-18,C,1510.0,33.0,34.5,25,0.156830,0.543453,0.005181,32610514,100,39,1510.119995,0.999921,33.75,29
2007-07-10,2007-08-18,C,1520.0,26.5,28.5,1548,0.150145,0.490355,0.005444,32598662,100,39,1510.119995,1.006543,27.50,29
2007-07-10,2007-08-18,C,1535.0,18.9,20.5,553,0.142940,0.405026,0.005559,32622229,100,39,1510.119995,1.016476,19.70,29
2007-07-10,2007-08-18,C,1540.0,16.5,18.1,1548,0.140027,0.375174,0.005553,32610513,100,39,1510.119995,1.019787,17.30,29
2007-07-10,2007-08-18,C,1550.0,12.3,13.6,520,0.134012,0.314218,0.005430,32590006,100,39,1510.119995,1.026409,12.95,29
2007-07-10,2007-08-18,C,1555.0,10.9,12.2,3325,0.134165,0.288535,0.005220,32657488,100,39,1510.119995,1.029719,11.55,29
2007-07-10,2007-08-18,P,1510.0,27.4,29.4,2974,0.162933,-0.456052,0.004989,32610515,100,39,1510.119995,0.999921,28.40,29
2007-07-10,2007-08-18,P,1520.0,31.4,33.4,2920,0.157791,-0.506876,0.005181,32598658,100,39,1510.119995,1.006543,32.40,29
2007-07-10,2007-08-18,P,1535.0,38.5,40.5,77,0.150709,-0.587875,0.005291,32622231,100,39,1510.119995,1.016476,39.50,29


In [10]:
# Group the data by date, strike price, and expiration date
grouped = data_1m.groupby(['Date', 'strike_price', 'Business_Days_to_Expiration'])

# Create straddles
straddles = []

for (date, strike_price, day_to_exp), group in grouped:
    if len(group) == 2:
        call = group[group['cp_flag'] == 'C']
        put = group[group['cp_flag'] == 'P']
        if not call.empty and not put.empty:
            straddle = {
                'Date': pd.to_datetime(date),
                'Strike_Price': strike_price,
                'D to Expiration': int(day_to_exp),
                'Call_Midprice': call['Midprice'].values[0],
                'Put_Midprice': put['Midprice'].values[0],
                'Total_Cost': call['Midprice'].values[0] + put['Midprice'].values[0],
                'Underlying Price': call['Close'].values[0],
                'Call_Delta': call['delta'].values[0],
                'Put_Delta': put['delta'].values[0],
                'Call_Optionid': call['optionid'].values[0],
                'Put_Optionid': put['optionid'].values[0],
                'Moneyness': call['Moneyness'].values[0]
            }
            straddles.append(straddle)

# Convert the list of straddles to a DataFrame
straddles_df = pd.DataFrame(straddles)
straddles_df['Date'] = pd.to_datetime(straddles_df['Date'])
straddles_df.set_index('Date', inplace=True)

In [11]:
for dates in straddles_df.index:
    if dates.day > 10:
        straddles_df.drop(dates, inplace=True, errors='ignore')
straddles_df

,Strike_Price,D to Expiration,Call_Midprice,Put_Midprice,Total_Cost,Underlying Price,Call_Delta,Put_Delta,Call_Optionid,Put_Optionid,Moneyness
Date,,,,,,,,,,,
2007-01-09,1425.0,29,15.40,22.85,38.25,1412.109985,0.455103,-0.539482,32304269,32304270,1.009128
2007-01-09,1430.0,29,13.00,25.60,38.60,1412.109985,0.410637,-0.579294,32309398,32309392,1.012669
2007-01-09,1435.0,29,10.80,28.40,39.20,1412.109985,0.365685,-0.619597,32309402,32309396,1.016210
2007-01-09,1440.0,29,8.80,31.40,40.20,1412.109985,0.320593,-0.659573,32303409,32303411,1.019751
2007-01-09,1445.0,29,7.10,34.60,41.70,1412.109985,0.277198,-0.698779,32303408,32303410,1.023291
...,...,...,...,...,...,...,...,...,...,...,...
2015-12-10,2010.0,26,69.60,29.15,98.75,2052.229980,0.644532,-0.353762,109020095,109020115,0.979422
2015-12-10,2015.0,26,65.65,30.60,96.25,2052.229980,0.630459,-0.368841,109020096,109020116,0.981859
2015-12-10,2040.0,26,49.00,38.55,87.55,2052.229980,0.547388,-0.451253,109020100,109020120,0.994041


In [12]:
straddles_df.loc['2015-09-02']

,Strike_Price,D to Expiration,Call_Midprice,Put_Midprice,Total_Cost,Underlying Price,Call_Delta,Put_Delta,Call_Optionid,Put_Optionid,Moneyness
Date,,,,,,,,,,,
2015-09-02,1880.0,27,97.25,36.40,133.65,1948.859985,0.664719,-0.331410,108286113,108286294,0.964667
2015-09-02,1885.0,27,93.55,37.70,131.25,1948.859985,0.653945,-0.342157,108286114,108286295,0.967232
2015-09-02,1920.0,22,64.50,42.60,107.10,1948.859985,0.576994,-0.419264,108226097,108226275,0.985191
2015-09-02,1920.0,27,69.20,48.25,117.45,1948.859985,0.569857,-0.425937,108286121,108286302,0.985191
2015-09-02,1925.0,22,61.15,44.25,105.40,1948.859985,0.562615,-0.433641,108226098,108226276,0.987757
2015-09-02,1925.0,27,65.90,49.90,115.80,1948.859985,0.556660,-0.439090,108286122,108286303,0.987757
2015-09-02,1930.0,22,57.95,46.15,104.10,1948.859985,0.547802,-0.448495,108226099,108226277,0.990323
2015-09-02,1940.0,22,51.70,49.75,101.45,1948.859985,0.517131,-0.479120,108226101,108226279,0.995454
2015-09-02,1945.0,22,48.65,51.70,100.35,1948.859985,0.501250,-0.495010,108226102,108226280,0.998019


In [13]:
# Reset index and create YearMonth column
straddles_reset = straddles_df.reset_index()
straddles_reset['YearMonth'] = straddles_reset['Date'].dt.to_period('M')

# Compute absolute moneyness differences
straddles_reset['Moneyness_Diff'] = abs(straddles_reset['Moneyness'] - 1)

# Sort by YearMonth, then by Moneyness_Diff, then by Date
straddles_reset.sort_values(['YearMonth', 'Moneyness_Diff', 'Date'], inplace=True)

# Select the first straddle (closest to 1 moneyness) for each month
first_straddles_daily = straddles_reset.drop_duplicates(subset='YearMonth', keep='first')

# Set Date as the index and drop unnecessary columns
first_straddles_daily.set_index('Date', inplace=True)
first_straddles_daily.drop(columns=['YearMonth', 'Moneyness_Diff'], inplace=True)

first_straddles_daily

C:\Users\benj0\AppData\Local\Temp\ipykernel_90480\1685846484.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_straddles_daily.drop(columns=['YearMonth', 'Moneyness_Diff'], inplace=True)


,Strike_Price,D to Expiration,Call_Midprice,Put_Midprice,Total_Cost,Underlying Price,Call_Delta,Put_Delta,Call_Optionid,Put_Optionid,Moneyness
Date,,,,,,,,,,,
2007-01-10,1425.0,28,16.50,20.65,37.15,1414.849976,0.481345,-0.516109,32304269,32304270,1.007174
2007-02-07,1450.0,28,19.95,14.50,34.45,1450.020020,0.578736,-0.428303,31916487,31916486,0.999986
2007-04-10,1450.0,29,21.50,19.20,40.70,1448.390015,0.549075,-0.454574,32430265,32430262,1.001112
2007-05-10,1490.0,27,28.45,18.90,47.35,1491.469971,0.568859,-0.429948,32485773,32485763,0.999014
2007-07-10,1510.0,29,33.75,28.40,62.15,1510.119995,0.543453,-0.456052,32610514,32610515,0.999921
...,...,...,...,...,...,...,...,...,...,...,...
2015-08-05,2100.0,27,26.30,31.35,57.65,2099.840088,0.477877,-0.519502,107969629,107969808,1.000076
2015-09-02,1950.0,22,45.75,53.90,99.65,1948.859985,0.485064,-0.511128,108226103,108226281,1.000585
2015-10-07,1995.0,22,35.50,38.25,73.75,1995.829956,0.496059,-0.501964,108554889,108555068,0.999584


In [26]:
def delta_hedging(straddles_df, transaction_cost=0):
    results = []

    # Precompute deltas for calls and puts
    call_deltas = option_df[option_df['cp_flag'] == 'C'].set_index('optionid')['delta']
    put_deltas = option_df[option_df['cp_flag'] == 'P'].set_index('optionid')['delta']

    # Align risk-free rates and prices with the full trading calendar
    full_calendar = pd.date_range(new_market_df.index.min(), new_market_df.index.max(), freq='B')
    sp_prices = new_market_df['Close'].reindex(full_calendar).fillna(method='ffill')
    rf_rates = new_market_df['RF'].reindex(full_calendar).fillna(method='ffill')

    for index, row in straddles_df.iterrows():
        strike_price = row['Strike_Price']
        call_id = row['Call_Optionid']
        put_id = row['Put_Optionid']

        # Extract delta series
        call_delta_series = call_deltas.loc[call_id]
        put_delta_series = put_deltas.loc[put_id]

        # Initial values
        total_cost = row['Total_Cost']
        underlying_price = row['Underlying Price']
        delta0 = -(row['Call_Delta'] + row['Put_Delta'])
        initial_cash = total_cost - delta0 * underlying_price

        # Initialize intermediate values
        prev_delta = delta0
        prev_prev_delta = 0  # No delta exists two periods before the start
        cash = initial_cash
        hedged_positions = []
        portfolio_values = []

        for i in range(int(row['D to Expiration'])):
            current_date = index + pd.offsets.BusinessDay(i)
            previous_date = index + pd.offsets.BusinessDay(i - 1) if i > 0 else None

            # Retrieve prices and deltas
            current_price = sp_prices.loc[current_date]  # Track the price for the current day
            rf = rf_rates.loc[current_date]
            current_call_delta = call_delta_series.get(current_date, 0)
            current_put_delta = put_delta_series.get(current_date, 0)
            curr_delta = -(current_call_delta + current_put_delta)

            # Update cash with the modified formula
            cash = (
                cash * np.exp(rf / 252) +
                (prev_delta - curr_delta) * current_price -
                transaction_cost * abs(prev_delta - prev_prev_delta) * current_price
            )
            hedged_position = curr_delta * current_price
            portfolio_value = cash + hedged_position

            # Store results
            hedged_positions.append(hedged_position)
            portfolio_values.append(portfolio_value)

            # Update deltas for the next iteration
            prev_prev_delta = prev_delta
            prev_delta = curr_delta

        # Calculate payoff for the short straddle at expiration
        payoff = max(0, strike_price - current_price) + max(0, current_price - strike_price)

        # Adjust the final value to account for the payoff
        final_value = portfolio_values[-1] - payoff

        results.append({
            'Date': index,
            'Initial Value': total_cost,
            'Final Value': final_value,
            'Hedged_Position': pd.DataFrame({'Date': full_calendar[:len(hedged_positions)], 'Hedged_Position': hedged_positions}),
            'Value': pd.DataFrame({'Date': full_calendar[:len(portfolio_values)], 'Value': portfolio_values})
        })

    return pd.DataFrame(results)

In [15]:
test_straddle = {
    'Strike_Price': 100,
    'Total_Cost': 10,  # Premium received
    'Underlying Price': 100,
    'Call_Delta': 0.5,
    'Put_Delta': -0.5,
    'D to Expiration': 30
}

# Test extreme price movements
extreme_prices = [50, 150]  # Far below and above the strike price

for price in extreme_prices:
    payoff = max(0, test_straddle['Strike_Price'] - price) + max(0, price - test_straddle['Strike_Price'])
    print(f"Price at Expiration: {price}, Payoff: {payoff}, P&L: {test_straddle['Total_Cost'] - payoff}")

Price at Expiration: 50, Payoff: 50, P&L: -40
Price at Expiration: 150, Payoff: 50, P&L: -40


In [28]:
# Delta Hedging without transaction costs
results_no_cost = delta_hedging(first_straddles_daily, transaction_cost=0)
results_no_cost
# results_no_cost[results_no_cost['Profit/Loss'] < 0]

# Plot histogram of P&L
# results_no_cost['Profit/Loss'].hist(bins=20, figsize=(10, 6), color='blue', alpha=0.7)
# plt.title('Distribution of P&L')
# plt.xlabel('Profit/Loss')
# plt.ylabel('Frequency')
# plt.axvline(0, color='red', linestyle='--', label='Break-Even')
# plt.legend()
# plt.grid(True)
# plt.show()

C:\Users\benj0\AppData\Local\Temp\ipykernel_90480\4196532604.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sp_prices = new_market_df['Close'].reindex(full_calendar).fillna(method='ffill')
C:\Users\benj0\AppData\Local\Temp\ipykernel_90480\4196532604.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rf_rates = new_market_df['RF'].reindex(full_calendar).fillna(method='ffill')


,Date,Initial Value,Final Value,Hedged_Position,Value
0,2007-01-10,37.15,6.805377,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 37.147...
1,2007-02-07,34.45,-28.375485,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 34.495...
2,2007-04-10,40.70,-31.781291,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 40.737...
3,2007-05-10,47.35,4.728391,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 47.399...
4,2007-07-10,62.15,-1.532786,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 62.186...
...,...,...,...,...,...
73,2015-08-05,57.65,-90.059961,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 57.65 1 2...
74,2015-09-02,99.65,73.469946,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 99.65 1 2...
75,2015-10-07,73.75,-31.179932,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 73.75 1 2...
76,2015-11-02,58.40,3.020951,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 58.400...


In [29]:
results_cost = delta_hedging(first_straddles_daily, transaction_cost=0.005)
results_cost

C:\Users\benj0\AppData\Local\Temp\ipykernel_90480\4196532604.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sp_prices = new_market_df['Close'].reindex(full_calendar).fillna(method='ffill')
C:\Users\benj0\AppData\Local\Temp\ipykernel_90480\4196532604.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rf_rates = new_market_df['RF'].reindex(full_calendar).fillna(method='ffill')


,Date,Initial Value,Final Value,Hedged_Position,Value
0,2007-01-10,37.15,6.309382,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 36.901...
1,2007-02-07,34.45,-30.566613,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 33.405...
2,2007-04-10,40.70,-33.153145,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 40.052...
3,2007-05-10,47.35,2.636308,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 46.363...
4,2007-07-10,62.15,-2.863538,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 61.527...
...,...,...,...,...,...
73,2015-08-05,57.65,-90.930631,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 57.212...
74,2015-09-02,99.65,72.961700,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 99.396...
75,2015-10-07,73.75,-31.298305,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 73.691...
76,2015-11-02,58.40,1.837962,Date Hedged_Position 0 2007-01-01 ...,Date Value 0 2007-01-01 57.809...
